In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
from multiprocessing import Process, Manager, current_process

In [198]:
def add_column(csv_name, time_target):
    df = pd.read_csv(f'csv/{time_target}/{csv_name}')
    df['date'] = csv_name[:8]
    return df
    
time_target = 'past'    
df = pd.concat(
#     [add_column(x, time_target) for x in os.listdir(f'csv/{time_target}') if 20 <= int(x[:2]) <= 26],
    [add_column(x, time_target) for x in os.listdir(f'csv/{time_target}') if 27 <= int(x[:2]) <= 31],
    ignore_index=True
) 
df 

,cur_match_h_team,cur_match_a_team,cur_match_score,cur_match_f_half,cur_match_s_half,h_match_h_team,h_match_a_team,h_match_score,h_match_f_half,h_match_s_half,a_match_h_team,a_match_a_team,a_match_score,a_match_f_half,a_match_s_half,date
0,Ботсвана,Экваториальная Гвинея,2-3,1 - 2,1 - 1,Экваториальная Гвинея,Ботсвана,2-0,1 - 0,1 - 0,Экваториальная Гвинея,Ботсвана,2-0,1 - 0,1 - 0,28_03_23
1,Намибия,Камерун,2-1,0 - 0,2 - 1,Камерун,Намибия,1-1,0 - 1,1 - 0,Камерун,Намибия,1-1,0 - 1,1 - 0,28_03_23
2,Эсватини,Кабо-Верде,0-1,0 - 0,0 - 1,Кабо-Верде,Эсватини,0-0,0 - 0,0 - 0,Кабо-Верде,Эсватини,0-0,0 - 0,0 - 0,28_03_23
3,Гамбия,Мали,1-0,0 - 0,1 - 0,Мали,Гамбия,2-0,1 - 0,1 - 0,Мали,Гамбия,2-0,1 - 0,1 - 0,28_03_23
4,Либерия,Южная Африка,1-2,1 - 1,0 - 1,Южная Африка,Либерия,2-2,2 - 0,0 - 2,Южная Африка,Либерия,2-2,2 - 0,0 - 2,28_03_23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,Лос Андес,Deportes Rengo,1-1,0 - 0,1 - 1,Депортес Консепсьон,Лос Андес,2-3,0 - 3,2 - 0,Deportes Rengo,Хенераль Веласкес,1-0,1 - 0,0 - 0,27_03_23
389,Рейнджерс (Ж),Селтик (Ж),1-1,1 - 0,0 - 1,Рейнджерс (Ж),Гамильтон (Ж),6-0,5 - 0,1 - 0,Хиберниан (Ж),Селтик (Ж),0-2,0 - 2,0 - 0,27_03_23
390,Макара,Cuniburo,0-0,0 - 0,0 - 0,Чакаритас,Макара,0-1,0 - 1,0 - 0,Cuniburo,Америка Кито,1-3,0 - 3,1 - 0,27_03_23
391,Портмор,Faulkland,4-0,2 - 0,2 - 0,Данбихолден,Портмор,1-3,0 - 0,1 - 3,Faulkland,Кавальер,2-2,1 - 0,1 - 2,27_03_23


In [199]:
def add_column(data, cur_team, target_team):
    rez = []
    for index in data.index:
        try:
            z_h, p_h = data[index].split('-')
            z_h, p_h = (int(z_h), int(p_h)) if cur_team[index] == target_team[index] else (int(p_h), int(z_h))

            rez.append(z_h - p_h) 
        except ValueError:
            rez.append(-999) 
    return rez


df['rez_h_f_half'] = add_column(
                                data=df['h_match_f_half'], 
                                cur_team=df['cur_match_h_team'], 
                                target_team=df['h_match_h_team']
                            )
df['rez_h_match'] = add_column(
                                data=df['h_match_score'], 
                                cur_team=df['cur_match_h_team'], 
                                target_team=df['h_match_h_team']
                            )

df['rez_a_f_half'] = add_column(
                                data=df['a_match_f_half'], 
                                cur_team=df['cur_match_a_team'], 
                                target_team=df['a_match_h_team']
                            )
df['rez_a_match'] = add_column(
                                data=df['a_match_score'], 
                                cur_team=df['cur_match_a_team'], 
                                target_team=df['a_match_h_team']
                            )


df['rez_c_f_half'] = add_column(
                                data=df['cur_match_f_half'], 
                                cur_team=df['cur_match_h_team'], 
                                target_team=df['cur_match_h_team']
                            )
df['rez_c_match'] = add_column(
                                data=df['cur_match_score'], 
                                cur_team=df['cur_match_h_team'], 
                                target_team=df['cur_match_h_team']
                            )
 

df_np = df[[
    'rez_h_f_half','rez_h_match',
    'rez_a_f_half', 'rez_a_match',
    'rez_c_f_half', 'rez_c_match',
]].to_numpy()
df_np

array([[-1, -2,  1,  2, -1, -1],
       [ 1,  0, -1,  0,  0,  1],
       [ 0,  0,  0,  0,  0, -1],
       ...,
       [ 1,  1, -3, -2,  0,  0],
       [ 0,  2,  1,  0,  2,  4],
       [-1,  0,  0,  0, -1, -1]])

In [206]:
lists = range(-5,5+1)

for h_f in lists:
    for h_score in lists:
        for a_f in lists:
            for a_score in lists:
                rez = df_np[ 
                    (df_np[:,0] == h_f) &
                    (df_np[:,1] == h_score) &
                    (df_np[:,2] == a_f) &
                    (df_np[:,3] == a_score) 
                ]
                
                if rez.shape[0] > 0:
                    r_h_f = rez[rez[:,4] >= 0].shape[0]
                    r_h_score = rez[rez[:,5] >= 0].shape[0]


                    r_a_f = rez[rez[:,4] <= 0].shape[0]
                    r_a_score = rez[rez[:,5] <= 0].shape[0]
                    proc_f = r_h_f/(r_h_f+r_a_f)
                    proc_score = r_h_score/(r_h_score+r_a_score)

#                     if proc_f >= 0.79 and (r_h_f+r_a_f) >= 5:
#                     if proc_f <= 0.21 and (r_h_f+r_a_f) >= 5:

                    if proc_score >= 0.79 and (r_h_score+r_a_score) >= 5:
#                     if proc_score <= 0.21 and (r_h_score+r_a_score) >= 5:
                        print(
                            f'{h_f:2d} {h_score:2d} {a_f:2d} {a_score:2d}'
                            f' ||| f_h =>{r_h_f:3d} {r_a_f:3d} = {proc_f:.3f}'
                            f' ||| score{r_h_score:3d} {r_a_score:3d}= {proc_score:.3f}')
                    
print('done')

 1  1  0 -1 ||| f_h =>  4   2 = 0.667 ||| score  4   1= 0.800
done


In [157]:
""" 20-26.03.23
-1 -2 -1  0 ||| f_h =>  4   3 = 0.571 ||| score  4   1= 0.800
 0 -2 -1 -1 ||| f_h =>  4   3 = 0.571 ||| score  4   1= 0.800
 0  1 -1  1 ||| f_h =>  4   5 = 0.444 ||| score  4   1= 0.800
 1 -1  0 -1 ||| f_h =>  7   1 = 0.875 ||| score  7   1= 0.875
 1  1  1  0 ||| f_h =>  3   1 = 0.750 ||| score  4   1= 0.800
"""
 

'\n-1 -2 -1  0 ||| f_h =>  4   3 = 0.571 ||| score  4   1= 0.800\n 0 -2 -1 -1 ||| f_h =>  4   3 = 0.571 ||| score  4   1= 0.800\n 0  1 -1  1 ||| f_h =>  4   5 = 0.444 ||| score  4   1= 0.800\n 1 -1  0 -1 ||| f_h =>  7   1 = 0.875 ||| score  7   1= 0.875\n 1  1  1  0 ||| f_h =>  3   1 = 0.750 ||| score  4   1= 0.800\n'

In [186]:
r = df_np[
    (df_np[:,0] == -1) &
    (df_np[:,1] == -2) &
    (df_np[:,2] == -1) &
    (df_np[:,3] == 0) 
]
r_h_f = r[r[:,4] >= 0]
r_h_score = r[r[:,5] >= 0]


r_a_f = r[r[:,4] <= 0]
r_a_score = r[r[:,5] <= 0]
r

array([[-1, -2, -1,  0,  2,  2],
       [-1, -2, -1,  0,  0, -1],
       [-1, -2, -1,  0,  0,  2],
       [-1, -2, -1,  0,  1,  3],
       [-1, -2, -1,  0, -1,  2]])

In [135]:
r_h_f.shape[0]

36

In [131]:
r_h_w = r[r[:,4] > 0].shape[0]
r_a_w = r[r[:,4] < 0].shape[0]
r_d = r[r[:,4] == 0].shape[0]

In [145]:
print(r_h_f.shape[0], r_a_f.shape[0])

36 33
